In [1]:
spark
sc
from pyspark.sql import functions as sf

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1646691297450_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Ejemeplo df=spark.read.csv('s3n://emontoya-datalake/datasets/sample_data.csv',inferSchema=True,header=True)
#Leer los datos desde s3
#df=spark.read.csv('s3://ojcontrerb-datasets/raw/Trabajo1_ST1800&ST18001/ds3_GlobalLandTemperaturesByCity/GlobalLandTemperaturesByCity.csv',inferSchema=True,header=True)
df=spark.read.csv('s3://ojcontrerb-datasets/Trabajo1/raw/global-land-temperatures/',inferSchema=True,header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Mostrar el esquema del Dataframe
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)

In [4]:
# Mostrar las dimensiones del dataframe
((df.count(),len(df.columns)))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(8599212, 7)

In [5]:
# Mostrar la información del dataframe
df.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+------------------+-----------------------------+--------+-----------+--------+---------+
|summary|        dt|AverageTemperature|AverageTemperatureUncertainty|    City|    Country|Latitude|Longitude|
+-------+----------+------------------+-----------------------------+--------+-----------+--------+---------+
|  count|   8599212|           8235082|                      8235082| 8599212|    8599212| 8599212|  8599212|
|   mean|      null|16.727432636250835|           1.0285747414536532|    null|       null|    null|     null|
| stddev|      null|10.353442482534422|           1.1297332887133706|    null|       null|    null|     null|
|    min|1743-11-01|-42.70399999999999|                        0.034|A Coruña|Afghanistan|   0.80N|    0.00W|
|    max|2013-09-01|            39.651|           15.395999999999999|  Ürümqi|   Zimbabwe|   8.84S|   99.91E|
+-------+----------+------------------+-----------------------------+--------+-----------+--------+---------+

In [6]:
# Agregamos una nueva columna con la Coordenada Geografica"
df2 = df.withColumn("GeographicCoordinate",sf.concat(sf.col("Latitude"),sf.lit(" "),sf.col("Longitude"))) 
df2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-----------------------------+-----+-------+--------+---------+--------------------+
|        dt| AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|GeographicCoordinate|
+----------+-------------------+-----------------------------+-----+-------+--------+---------+--------------------+
|1743-11-01|              6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1743-12-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-01-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-02-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-03-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-04-01| 5.7879999999999985|           3.6239999999999997|År

In [7]:
#Removemos todas las filas que tengan algun valor nulo en cualquiera de sus columnas
df2 = df2.na.drop()
df2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-----------------------------+-----+-------+--------+---------+--------------------+
|        dt| AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|GeographicCoordinate|
+----------+-------------------+-----------------------------+-----+-------+--------+---------+--------------------+
|1743-11-01|              6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-04-01| 5.7879999999999985|           3.6239999999999997|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-05-01|             10.644|           1.2830000000000001|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-06-01| 14.050999999999998|                        1.347|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-07-01|             16.082|                        1.396|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-09-01| 12.780999999999999|                        1.454|År

In [8]:
#Calculamos el numero de filas removidas
df.count()-df2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

364130

In [13]:
#Insertamos la tabla 
#uri_output='s3://ojcontrerb-datasets/trusted/trabajo1/Global/GlobalLandTemperaturesByCity_Cleaned'
uri_output='s3://ojcontrerb-datasets/Trabajo1/trusted/Global/GlobalLandTemperaturesByCity_Cleaned'
df2.coalesce(1).write.format("csv").option("header","true").save(uri_output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#Comprobamos leyendo el archivo en la zona trusted
#df_cleaned=spark.read.csv('s3://ojcontrerb-datasets/trusted/trabajo1/Global/GlobalLandTemperaturesByCity_Cleaned/part-00000-05431328-0e33-45f6-9504-d4a8e0c82bc9-c000.csv',inferSchema=True,header=True)
df_cleaned=spark.read.csv('s3://ojcontrerb-datasets/Trabajo1/trusted/Global/GlobalLandTemperaturesByCity_Cleaned/part-00000-ffc58119-0f6b-4340-8f8c-fb90c3b30a14-c000.csv',inferSchema=True,header=True)
df_cleaned.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-----------------------------+-----+-------+--------+---------+--------------------+
|        dt| AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|GeographicCoordinate|
+----------+-------------------+-----------------------------+-----+-------+--------+---------+--------------------+
|1743-11-01|              6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-04-01| 5.7879999999999985|           3.6239999999999997|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-05-01|             10.644|           1.2830000000000001|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-06-01| 14.050999999999998|                        1.347|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-07-01|             16.082|                        1.396|Århus|Denmark|  57.05N|   10.33E|       57.05N 10.33E|
|1744-09-01| 12.780999999999999|                        1.454|År

In [15]:
#Realizamos una transformación que nos permitirá realizar un análisis exploratorio de datos
df_refined = df_cleaned.groupBy("Country","City").mean("AverageTemperature")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#Mostramos el nuevo dataframe con la información transformada
df_refined.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+-----------------------+
|             Country|      City|avg(AverageTemperature)|
+--------------------+----------+-----------------------+
|           Argentina|   Cordoba|     16.913611578947396|
|            Pakistan|   Chiniot|     24.138396543446955|
|          Madagascar| Antsirabe|     19.815439024390216|
|           Indonesia|  Banjaran|     24.768451303885037|
|       Côte D'Ivoire|   Abidjan|     26.163737197524014|
|             Nigeria|   Amaigbo|      26.61282420749278|
|Bosnia And Herzeg...|Banja Luka|      9.081287428932413|
|               India|  Ambattur|     28.417858452950554|
|       United States|      Cary|      14.41886117345303|
|               Chile|     Arica|     19.173233684210505|
|          Kazakhstan|    Atyrau|       8.06841197046249|
|               Haiti| Carrefour|     26.468403139013443|
|               India|  Azamgarh|      25.11258609668402|
|           Indonesia|   Cilegon|     26.508209686003163|
|           In

In [17]:
#Escribimos el nuevo dataframe en la zona refined del datalake
#uri_output='s3://ojcontrerb-datasets/refined/trabajo1/Global/GlobalLandTemperaturesByCity_Refined_GroupBy'
uri_output='s3://ojcontrerb-datasets/Trabajo1/refined/Global/GlobalLandTemperaturesByCity_Refined_GroupBy'
df_refined.coalesce(1).write.format("csv").option("header","true").save(uri_output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#Comprobamos leyendo el archivo en la zona refined
df_analysis = spark.read.csv('s3://ojcontrerb-datasets/Trabajo1/refined/Global/GlobalLandTemperaturesByCity_Refined_GroupBy/part-00000-e7fefe16-d827-4e40-bf53-bc8e663b2681-c000.csv',inferSchema=True,header=True)
df_analysis.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+-----------------------+
|             Country|      City|avg(AverageTemperature)|
+--------------------+----------+-----------------------+
|           Argentina|   Cordoba|     16.913611578947396|
|            Pakistan|   Chiniot|     24.138396543446955|
|          Madagascar| Antsirabe|     19.815439024390216|
|           Indonesia|  Banjaran|     24.768451303885037|
|       Côte D'Ivoire|   Abidjan|     26.163737197524014|
|             Nigeria|   Amaigbo|      26.61282420749278|
|Bosnia And Herzeg...|Banja Luka|      9.081287428932413|
|               India|  Ambattur|     28.417858452950554|
|       United States|      Cary|      14.41886117345303|
|               Chile|     Arica|     19.173233684210505|
|          Kazakhstan|    Atyrau|       8.06841197046249|
|               Haiti| Carrefour|     26.468403139013443|
|               India|  Azamgarh|      25.11258609668402|
|           Indonesia|   Cilegon|     26.508209686003163|
|           In

In [19]:
df_analysis.orderBy(df_analysis["avg(AverageTemperature)"].desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------+-----------------------+
|  Country|        City|avg(AverageTemperature)|
+---------+------------+-----------------------+
| Djibouti|      Jibuti|     29.152790108564506|
|    Sudan|    Khartoum|     29.081291289592794|
|    Sudan|  Umm Durman|     29.081291289592794|
|    Niger|      Niamey|     29.062560090702984|
|    Sudan|     Kassala|     28.938775931232133|
|     Mali|       Mopti|      28.78444858817257|
|    Sudan|       Rabak|     28.598225729600962|
|    Sudan|       Kusti|     28.598225729600962|
|    India|       Avadi|     28.417858452950554|
|    India|    Ambattur|     28.417858452950554|
|    India|      Madras|     28.417858452950554|
|    India|    Tambaram|     28.417858452950554|
|    India|  Pallavaram|     28.417858452950554|
|    India|Tiruvottiyur|     28.417858452950554|
|    India| Kanchipuram|     28.417858452950554|
|    India|    Tirupati|     28.417858452950554|
|    India|     Alandur|     28.417858452950554|
|  Nigeria|      Sok

In [20]:
df_analysis.orderBy(df_analysis["avg(AverageTemperature)"].asc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------------+-----------------------+
|      Country|               City|avg(AverageTemperature)|
+-------------+-------------------+-----------------------+
|       Russia|            Norilsk|     -11.85474967630557|
|       Russia|              Kyzyl|    -6.2224518774277255|
|       Russia|              Chita|     -4.360299956840743|
|       Russia|         Ust Ilimsk|    -3.9967997410444496|
|       Russia|       Nefteyugansk|    -3.5382811078958243|
|       Russia|             Surgut|    -3.5382811078958243|
|     Mongolia|        Ulaanbaatar|    -3.3654853195164023|
|       Russia|           Ulan Ude|     -3.179915407854988|
|       Russia|             Bratsk|    -2.8179848942598165|
|        China|             Hailar|    -2.5129801467414716|
|        China|            Yakeshi|    -2.5129801467414716|
|       Russia|Komsomolsk Na Amure|    -2.3709624773960196|
|       Russia|     Nizhnevartovsk|     -2.337914501257337|
|United States|          Anchorage|    -